In [1]:
from google.colab import drive

drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
data = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/self_drive_car/dataset/HateSpeechDataset.csv")
text_df = pd.read_csv("/content/gdrive/MyDrive/Colab Notebooks/self_drive_car/dataset/fake_or_real_news.csv")

In [4]:
df = data[['Content', 'Label']]
df = df[df['Label'] != 'Label']
df.reset_index(drop=True, inplace=True)

In [5]:
text = list(text_df.text.values)
joined_text = ' '.join(text)
pratical_text = joined_text[:10000]
print(pratical_text[:100])

Daniel Greenfield, a Shillman Journalism Fellow at the Freedom Center, is a New York writer focusing


In [6]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
stopwords = set(stopwords.words('english'))
print(stopwords)
lemmatizer = WordNetLemmatizer()
import string
def text_cleaning(a, remove_stopwords=True):
    # Noktalama işaretlerini kaldır
    remove_punc = [char for char in a if char not in string.punctuation]
    remove_punc = ''.join(remove_punc)

    # Kelimeleri tokenlara ayır ve stopwords'leri çıkar (opsiyonel)
    tokens = word_tokenize(remove_punc)
    if remove_stopwords:
        cleaned_tokens = [word for word in tokens if word.lower() not in stopwords]
    else:
        cleaned_tokens = tokens

    # Lemmatize et
    lemmatized_tokens = [lemmatizer.lemmatize(word.lower()) for word in cleaned_tokens]

    return lemmatized_tokens
df.iloc[:,0].apply(text_cleaning)
from sklearn.model_selection import train_test_split

X = df['Content']
y = df['Label']
# feature transformation - TF-IDF to the processed text feature
from sklearn.feature_extraction.text import TfidfVectorizer

# initialize the TfidfVectorizer  object
vectorizer = TfidfVectorizer()

# vectorize and perform TF-IDF to the training data
X_vectorized = vectorizer.fit_transform(X)
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
best_accuracy = 0
best_y_pred = None
best_y_test = None

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Loop through each split and fit the model
for train_index, test_index in skf.split(X_vectorized, y):
    X_train, X_test = X_vectorized[train_index], X_vectorized[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # Fit the model
    clf.fit(X_train, y_train)

    # Make predictions
    y_pred = clf.predict(X_test)

    # Calculate accuracy
    accuracy = accuracy_score(y_test, y_pred)

    # Check if current accuracy is the best
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_y_pred_temp = y_pred
        best_y_test_temp = y_test

# Update best_y_pred and best_y_test with the ones associated with the best accuracy
best_y_pred = best_y_pred_temp
best_y_test = best_y_test_temp

unique_labels_test = np.unique(best_y_test)
unique_labels_pred = np.unique(best_y_pred)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


{'theirs', 'your', 'in', 'above', 'some', 't', 'during', 'below', 'i', 'it', "needn't", 'each', 'had', 'yourself', "you've", 'very', 'we', 'to', 'themselves', 'who', 'can', 'most', "wouldn't", 'which', 'they', 'am', 'he', 'did', 'at', 'herself', 'not', 'this', 's', 'just', 'mustn', 're', 'further', 'doesn', 'his', "she's", 'with', 'isn', 'whom', 'all', 'under', 'a', 'and', 'does', 'been', 'of', 'only', 'she', 'being', 'shouldn', 'were', 'own', 'nor', 'yours', 'them', "hasn't", "won't", 'its', 'while', 'what', 'him', 'having', "shan't", 've', 'y', 'the', 'now', 'm', 'because', 'for', 'any', 'was', 'than', "shouldn't", "should've", 'be', "wasn't", 'o', 'ma', 'needn', 'about', 'shan', 'here', "you're", 'if', 'where', 'couldn', 'mightn', 'hers', "aren't", "hadn't", 'her', 'same', 'too', "you'd", 'should', 'once', 'wouldn', 'weren', 'are', 'between', 'few', 'other', 'my', 'but', 'do', 'into', 'itself', 'doing', 'aren', "doesn't", 'yourselves', 'is', 'has', 'before', 'down', 'you', 'there', 

In [7]:
from nltk.tokenize import RegexpTokenizer
Tokenizer = RegexpTokenizer(r'\w+')
tokens =  Tokenizer.tokenize(pratical_text.lower())
unique_tokens = np.unique(tokens)
unique_token_index = {token: index for index, token in enumerate(unique_tokens)}
n_words = 10
input_words = []
next_words = []
for i in range(len(tokens) - n_words):
    input_words.append(tokens[i:i + n_words])
    next_words.append(tokens[i + n_words])


In [8]:
from tensorflow.keras.models import load_model
model = load_model("/content/model.h5")

In [9]:
import random
def predict_next_words(input_text, n_best):
  input_text = input_text.lower()
  X = np.zeros((1, n_words, len(unique_tokens)))
  for i, word in enumerate(input_text.split()):
    X[0, i, unique_token_index[word]] = 1

  predictions = model.predict(X)[0]
  return np.argpartition(predictions, -n_best)[-n_best:]
def generate_text(input_text, text_length, creativity=3):
  word_sequence = input_text.split()
  current=0
  for _ in range(text_length):
    sub_sequence = " ".join(Tokenizer.tokenize( " ".join(word_sequence).lower())[current:current+n_words])
    try:
      choice = unique_tokens[random.choice(predict_next_words(sub_sequence, creativity))]
    except:
       choice = random.choice(unique_tokens)
    word_sequence.append(choice)
    current += 1
  return " ".join(word_sequence)

In [10]:
def predict_label(sentence):
    cleaned_sentence = text_cleaning(sentence)
    print(cleaned_sentence)
    vectorized_sentence = vectorizer.transform(cleaned_sentence)  # Vektörleştirmede liste kullanıldı
    print(vectorized_sentence)
    prediction = clf.predict(vectorized_sentence)
    print(prediction)

    # Sansürlü kelimeyi bul
    censored_words = [cleaned_sentence[index] for index, pred in enumerate(prediction) if pred == "1"]
    print("Censored words:", censored_words)

    # Eğer censored_words boşsa, temizlenmiş cümleyi direkt olarak döndür
    if not censored_words:
        return " ".join(cleaned_sentence)

    # Sansürlü kelimeleri sentence içinden sil
    sanitized_sentence = sentence
    for word in censored_words:
        sanitized_sentence = sanitized_sentence.replace(word, "")

    return sanitized_sentence

In [11]:
def results(sentence):
  answer = predict_label(sentence)
  answer = answer + "insulting word"
  gtext = generate_text(answer, 20,20)
  print(gtext)

In [ ]:
results("As the discussion reached its climax, the atmosphere in the hall became tense. Suddenly, one of the speakers shouted bastard")

['discussion', 'reached', 'climax', 'atmosphere', 'hall', 'became', 'tense', 'suddenly', 'one', 'speaker', 'shouted', 'bastard']
  (0, 29604)	1.0
  (1, 93706)	1.0
  (2, 20518)	1.0
  (3, 7061)	1.0
  (4, 47222)	1.0
  (5, 9882)	1.0
  (6, 114606)	1.0
  (7, 109920)	1.0
  (8, 80842)	1.0
  (9, 106686)	1.0
  (10, 103178)	1.0
  (11, 9399)	1.0
['0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '0' '1']
Censored words: ['bastard']
As the discussion reached its climax, the atmosphere in the hall became tense. Suddenly, one of the speakers shouted insulting word allowed suddenly belief didn assaults taking times been proved didn re try ringing now out new hope j unite substance
